# Generate Passport Index datasets
* Data by Passport Index 2025: https://www.passportindex.org/
* In both tidy and matrix formats
* Using ISO-2, ISO-3, and full country names

In [1]:
import pandas as pd
import json
import time

# Load tqdm!
from tqdm import tqdm
tqdm.pandas()

### In your browser,

1. Navigate to https://www.passportindex.org/comparebyPassport.php?p1=sa&y1=2024
1. Open Dev tools > Network
1. Force-reload the page (e.g. Cmd+Shift+R)
1. Find a POST request to *compare2.php*, right-click, and select 'Copy Value' -> 'Copy as cURL'
1. Paste below

In [3]:
curl_command = """
curl 'https://www.passportindex.org/incl/compare2.php' --compressed -X POST -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:133.0) Gecko/20100101 Firefox/133.0' -H 'Accept: */*' -H 'Accept-Language: en-GB,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br, zstd' -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' -H 'X-Requested-With: XMLHttpRequest' -H 'Origin: https://www.passportindex.org' -H 'DNT: 1' -H 'Sec-GPC: 1' -H 'Connection: keep-alive' -H 'Referer: https://www.passportindex.org/comparebyPassport.php?p1=sa&y1=2024' -H 'Cookie: _ga_7Z48X951ET=GS1.1.1735809124.8.0.1735809133.0.0.0; _ga=GA1.1.982642551.1730071472; PHPSESSID=0833ebcdbd2b18a09ae244ed8b0328e8; cf_clearance=LGqan9v3k9U3GBVjE2NDKWl3d2TrCGOUMvvWRu3NBXY-1735809123-1.2.1.1-aD3.TI6s20.jSzc0AzJRZYFO8C0lnoXKbCBQebFQkmUwW3ATdyzZ84p6L3zk8F9Z9HOmbsTNxoHUicQBjiCu3au_dBv1ExLUg.hcpQSv.Bc5FYsyZ6DaCUHVbOU.h5u9B2FP29oCZBfr8J4mgmYSfdjWEn81.Th1fR9TgbOrdgkKaKjgaDcDF3KnWg83S3WXN_Tyhoe7_vUUntHYJ.PLGfZ_uzQH1a5kPOqTrUQMKOljiLPuvUHdn0XeRkrkEJ7i2edeeqadOftFKS8D2BlVZyKZBKXmdXH8Dy_KLYs.5gWz50gSwRLkCCYHxBpm_LWBI4YvZjQfD1UGVm0jdU.225fbHEGwNrZXnwhqjiZ1tLc' -H 'Sec-Fetch-Dest: empty' -H 'Sec-Fetch-Mode: cors' -H 'Sec-Fetch-Site: same-origin' -H 'TE: trailers' --data-raw 'compare=3&year=2024&cc=sa&csrf_token=075fdd79354a838eb88fb140268ef94b70e134749661ac038d66e48ca72460ab'"""

In [4]:
# Read all country codes

country_data = (
    pd.read_csv(
        'https://gist.githubusercontent.com/ilyankou/b2580c632bdea4af2309dcaa69860013/raw/420fb417bcd17d833156efdf64ce8a1c3ceb2691/country-codes',
        dtype=str
    )
    .fillna('NA')
)

In [5]:
def get_data(country_code):

    time.sleep(6.1)

    curl_command_ = (curl_command
                     .replace('curl ', 'curl -s ')
                     .replace(', br, zstd', '')
                     .replace('cc=sa', f'cc={str.lower(country_code)}')
                    )

    res = !{curl_command_}
    return json.loads(res[0])

In [6]:
country_data['travel'] = country_data.ISO2.progress_apply(get_data)

100%|█████████████████████████████████████████| 199/199 [20:58<00:00,  6.32s/it]


## Get data from PassportIndex

In [7]:
assert country_data['travel'].isna().sum() == 0, \
    f"Didn't fetch {country_data['travel'].isna().sum()} rows!"

## Clean up the data

In [8]:
obj = {}

for passport in country_data.ISO2.tolist():
    
    # Add passport to the object
    if passport not in obj:
        obj[passport] = {}
    
    # Add destinations for the given passport
    for dest in country_data.query(f'ISO2 == "{passport}"').iloc[0]['travel']:
        
        text = dest['text'].lower()
        res = ''
        
        # ** Visa required, incl Cuba's tourist card or China's Exit Entry Permit for Macau/HK **
        if text == 'visa required' or text == 'tourist card' or text == 'exit-entry permit':
            res = 'visa required'
        
        # ** Visa on arrival **
        elif 'visa on arrival' in text:
            res = 'visa on arrival'
            
        # ** Covid-19 ban ** 
        elif text == 'covid-19 ban':
            res = 'covid ban'
            
        # ** Visa-free, incl. Seychelles' tourist registration **
        elif 'visa-free' in text or 'tourist registration' in text or 'visa waiver' in text:
            res = dest['dur'] if dest['dur'] != '' else 'visa free'
            
        # ** eVisas, incl eVisitors (Australia), eTourist cards (Suriname),
        # eTA (US), and pre-enrollment (Ivory Coast), or EVW (UK) **
        elif 'evis' in text or 'etourist' in text or text == 'eta' or text == 'pre-enrollment' or text == 'evw':
            res = 'e-visa'
            
        # ** No admission, including Trump ban **
        elif text == 'trump ban' or text == 'not admitted':
            res = 'no admission'

        elif 'arrival card' in text or 'e-ticket' in text:
            res = 'visa free'
        
        # Update the result!
        obj[passport][ dest['code'] ] = res if res != '' else dest['text']

In [9]:
for x, y in country_data.head(5).iterrows():
    print(x, y['ISO3'])

0 AFG
1 ALB
2 DZA
3 AND
4 AGO


## Save

In [10]:
# ISO-2: Matrix
matrix = pd.DataFrame(obj).T.fillna(-1)
matrix.to_csv('passport-index-matrix-iso2.csv', index_label='Passport')

# ISO-2: Tidy
matrix.stack().to_csv(
    'passport-index-tidy-iso2.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])

# ISO-3: Matrix
iso2to3 =  { y['ISO2']:y['ISO3'] for _,y in country_data.iterrows() }
matrix.rename(columns=iso2to3, index=iso2to3).to_csv('passport-index-matrix-iso3.csv', index_label='Passport')

# ISO-3: Tidy
matrix.rename(columns=iso2to3, index=iso2to3).stack().to_csv(
    'passport-index-tidy-iso3.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])


# Country names: Matrix
iso2name =  { y['ISO2']:y['Country'] for _,y in country_data.iterrows() }
matrix.rename(columns=iso2name, index=iso2name).to_csv('passport-index-matrix.csv', index_label='Passport')

# Country names: Tidy
matrix.rename(columns=iso2name, index=iso2name).stack().to_csv(
    'passport-index-tidy.csv',
    index_label=['Passport', 'Destination'],
    header=['Requirement'])

In [11]:
# Print all values
tidy = matrix.rename(columns=iso2to3, index=iso2to3).stack()
tidy.value_counts()

visa required      12921
90                  7651
e-visa              7523
visa on arrival     6053
visa free           1883
30                  1698
180                  685
120                  233
-1                   199
60                   192
14                   107
21                   106
360                   97
15                    94
42                    60
no admission          34
28                    23
240                   15
45                    15
10                     6
7                      4
150                    1
31                     1
Name: count, dtype: int64

In [12]:
#tidy[tidy == "Exit-entry Permit"]

In [13]:
# Which countries have no admission? (Typically war zones etc)
tidy[ tidy == 'no admission' ]

ARM  AZE    no admission
BGD  IRQ    no admission
     LBY    no admission
HTI  SUR    no admission
IRN  LBY    no admission
ISR  DZA    no admission
     BGD    no admission
     BRN    no admission
     IRN    no admission
     LBN    no admission
     LBY    no admission
     MYS    no admission
     PAK    no admission
     SAU    no admission
     SYR    no admission
     YEM    no admission
XKX  ARM    no admission
     KHM    no admission
     CUB    no admission
     HKG    no admission
     SYC    no admission
PRK  JPN    no admission
     LKA    no admission
PAK  LBY    no admission
PSE  MDG    no admission
     SYR    no admission
PHL  KWT    no admission
SOM  AUS    no admission
     CAN    no admission
SDN  LBY    no admission
SYR  LBY    no admission
TWN  GEO    no admission
TJK  KGZ    no admission
YEM  LBY    no admission
dtype: object

In [14]:
assert tidy.loc[('BLR', 'RUS')] == '90', "Check data!"

### Difference with previous run
* Typically the difference between two consecutive months would be in low hundreds.
* If difference is higher, double-check the data!

In [15]:
x = (tidy
 .reset_index()
 .rename(
     columns={'level_0': 'Passport', 'level_1': 'Destination', 0: 'Requirement'}
 )
 .merge(
     pd.read_csv('legacy/2024-12-04/passport-index-tidy-iso3.csv'), # SELECT VERSION TO COMPARE!
     how='left',
     left_on=['Passport', 'Destination'],
     right_on=['Passport', 'Destination'],
     suffixes=('_new', '_old')
 )
 .assign(
     is_different=lambda df_: df_.Requirement_old.ne(df_.Requirement_new)
 )
 .query('is_different & (Passport != Destination)')
 .drop(columns=['is_different'])
)

x

,Passport,Destination,Requirement_new,Requirement_old
241,ALB,CUB,e-visa,visa required
440,DZA,CUB,e-visa,visa required
639,AND,CUB,e-visa,visa required
1182,ATG,GBR,e-visa,180
1381,ARG,GBR,e-visa,180
...,...,...,...,...
38250,VUT,CUB,e-visa,visa required
38449,VAT,CUB,e-visa,visa required
38648,VEN,CUB,e-visa,visa required
39444,ZWE,CUB,e-visa,visa required


In [16]:
# What changed for the UK?
x.query('Passport == "GBR"')

,Passport,Destination,Requirement_new,Requirement_old


In [23]:
x.query('Destination == "GBR"')

,Passport,Destination,Requirement_new,Requirement_old
1182,ATG,GBR,e-visa,180
1381,ARG,GBR,e-visa,180
1779,AUS,GBR,e-visa,180
2376,BHS,GBR,e-visa,180
2973,BRB,GBR,e-visa,180
3570,BLZ,GBR,e-visa,180
4565,BWA,GBR,e-visa,180
4764,BRA,GBR,e-visa,180
4963,BRN,GBR,e-visa,180
6157,CAN,GBR,e-visa,180
